In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.9/429.9 kB 25.7 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: accelerate 1.11.0
Uninstalling accelerate-1.11.0:
  Successfully uninstalled accelerate-1.11.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 27.1 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, set_seed
from datasets import  load_dataset , load_from_disk
import matplotlib.pyplot as plt
import pandas as pd
# from datasets import load_dataset , load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")
from tqdm import tqdm
import torch

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
model_mail = AutoModelForSeq2SeqLM.from_pretrained(model).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install huggingface_hub

from huggingface_hub import login
login()


In [ ]:
from huggingface_hub import login
from google.colab import userdata

try:
    hf_token = userdata.get('HF_TOKEN')
    login(token=hf_token)
    print("Successfully logged in to Hugging Face Hub.")
except Exception as e:
    print(f"Could not log in to Hugging Face Hub: {e}")
    print("Please ensure your HF_TOKEN is correctly set in Colab secrets.")

Could not log in to Hugging Face Hub: Secret HF_TOKEN does not exist.
Please ensure your HF_TOKEN is correctly set in Colab secrets.


In [ ]:
dataset_mail = load_dataset("cnn_dailymail", '3.0.0')
dataset_mail

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [ ]:
dataset_mail

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [ ]:
dataset_mail["train"]["highlights"][1]

'Mentally ill inmates in Miami are housed on the "forgotten floor"\nJudge Steven Leifman says most are there as a result of "avoidable felonies"\nWhile CNN tours facility, patient shouts: "I am the son of the president"\nLeifman says the system is unjust and he\'s fighting for change .'

In [ ]:
dataset_mail["train"]["article"][1]

'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to foll

In [ ]:
dataset_mail["train"][1]["highlights"]

'Mentally ill inmates in Miami are housed on the "forgotten floor"\nJudge Steven Leifman says most are there as a result of "avoidable felonies"\nWhile CNN tours facility, patient shouts: "I am the son of the president"\nLeifman says the system is unjust and he\'s fighting for change .'

In [ ]:

split_lengths = [len(dataset_mail[split])for split in dataset_mail.keys()]
print(f"Split lengths: {split_lengths}")

print(f"Features: {dataset_mail['train'].column_names}")
print(dataset_mail["test"][1]["highlights"])
print("\nSummary")
print(dataset_mail["test"][1]["article"])

Split lengths: [287113, 13368, 11490]
Features: ['article', 'highlights', 'id']
Theia, a bully breed mix, was apparently hit by a car, whacked with a hammer and buried in a field .
"She's a true miracle dog and she deserves a good life," says Sara Mellado, who is looking for a home for Theia .

Summary
(CNN)Never mind cats having nine lives. A stray pooch in Washington State has used up at least three of her own after being hit by a car, apparently whacked on the head with a hammer in a misguided mercy killing and then buried in a field -- only to survive. That's according to Washington State University, where the dog -- a friendly white-and-black bully breed mix now named Theia -- has been receiving care at the Veterinary Teaching Hospital. Four days after her apparent death, the dog managed to stagger to a nearby farm, dirt-covered and emaciated, where she was found by a worker who took her to a vet for help. She was taken in by Moses Lake, Washington, resident Sara Mellado. "Conside

NOW WE NEED TO CONVERT THIS WORDS INTO VECTOR
SO WE NEED TO CONVERT INTO VECTORS
WE WILL USE TOKENIZER

In [ ]:
def convert_examples_to_features(example_batch):
  input = tokenizer(example_batch['article'],max_length=100,truncation=True)
  with tokenizer.as_target_tokenizer():
    target = tokenizer(example_batch['highlights'],max_length=64,truncation=True)

  return{
      'input_ids':input['input_ids'],
      'attention_mask':input['attention_mask'],
      'labels':target['input_ids']
  }

In [ ]:
dataset_mapped = dataset_mail.map(convert_examples_to_features,batched=True)

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
dataset_mapped["train"]

Dataset({
    features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 287113
})

In [ ]:
dataset_mapped["train"]["input_ids"][1]

[8088,
 131,
 116,
 1351,
 151,
 222,
 150,
 16651,
 109,
 48787,
 679,
 108,
 11869,
 21089,
 116,
 537,
 153,
 1747,
 115,
 3693,
 990,
 111,
 5935,
 109,
 1357,
 893,
 109,
 702,
 107,
 1063,
 108,
 23893,
 21276,
 1141,
 131,
 15991,
 839,
 780,
 865,
 114,
 7194,
 241,
 223,
 113,
 109,
 19105,
 127,
 10421,
 5313,
 107,
 983,
 36117,
 12771,
 124,
 109,
 198,
 67262,
 1030,
 745,
 241,
 223,
 10421,
 5313,
 19105,
 127,
 12771,
 115,
 4977,
 269,
 2498,
 107,
 7201,
 39134,
 108,
 1967,
 143,
 40155,
 158,
 1315,
 139,
 13487,
 1030,
 113,
 109,
 4977,
 121,
 38654,
 81367,
 15250,
 1944,
 117,
 15559,
 109,
 198,
 67262,
 1030,
 496,
 1063,
 108,
 1]

In [ ]:
dataset_mapped["train"]["labels"][1]

[11390,
 445,
 5313,
 19105,
 115,
 4977,
 127,
 12771,
 124,
 109,
 198,
 67262,
 1030,
 194,
 8260,
 8950,
 86580,
 1121,
 649,
 205,
 127,
 186,
 130,
 114,
 711,
 113,
 198,
 55197,
 1431,
 67429,
 194,
 1041,
 11869,
 4593,
 1944,
 108,
 1532,
 45259,
 151,
 198,
 187,
 346,
 109,
 1601,
 113,
 109,
 1977,
 194,
 86580,
 1121,
 649,
 109,
 327,
 117,
 34348,
 111,
 178,
 131,
 116,
 3780,
 118,
 411,
 110,
 1]

In [ ]:
dataset_mapped["train"]["attention_mask"][1]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

TRAINING

In [ ]:
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model=model_mail)

In [ ]:
from transformers import TrainingArguments, Trainer
trainer_args = TrainingArguments(
    output_dir='cnn-mail',num_train_epochs=1,warmup_steps=500,
    per_device_train_batch_size=1 , per_device_eval_batch_size = 1 ,
    weight_decay = 0.01 , logging_steps = 10 ,
    # evaluation_strategy = 'steps', eval_steps = 500 ,  # Temporarily removed to resolve TypeError
    save_steps = 1e6,
    gradient_accumulation_steps = 16
)

In [ ]:
trainer = Trainer( model = model_mail , args = trainer_args ,
                  tokenizer = tokenizer , data_collator = seq2seq_data_collator ,
                  train_dataset = dataset_mapped["train"],
                  eval_dataset = dataset_mapped["validation"])

/tmp/ipython-input-3023603706.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer( model = dataset_mail , args = trainer_args ,


AttributeError: 'DatasetDict' object has no attribute 'to'

In [ ]:
trainer.train()

EVALUATION

In [ ]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """
    Split the dataset into smaller batches that we can process simultaneously.
    Yield successive batch-sized chunks from list_of_elements.
    """
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]


def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                                batch_size=16, device=device,
                                column_text="article",
                                column_summary="highlights"):

    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                   padding="max_length", return_tensors="pt")

        summaries = model.generate(
        input_ids=inputs["input_ids"].to(device),
        attention_mask=inputs["attention_mask"].to(device),
        length_penalty=0.8, num_beams=8, max_length=128)


        decoded_summaries = [tokenizer.decode(
                        s, skip_special_tokens=True,
                        clean_up_tokenization_spaces=True)
                     for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    score = metric.compute()
    return score



In [ ]:
rouge_names = ["rouge1" , "rouge2" , "rougeL" , "rougeLsum"]
rouge_metric = load_metric('rouge')

In [ ]:
score = calculate_metric_on_test_ds(
    dataset_mail['test'],
    rouge_metric,
    trainer.model,
    tokenizer,
    batch_size = 2,
    column_text = 'article',
    column_summary= 'highlights'
)
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )
pd.DataFrame(rouge_dict, index = [f'mail'])

SAVE MODEL

In [ ]:
model_mail.save.pretrained("cnn_dailymail")

In [ ]:
tokenizer.save_pretrained("tokenizer")

PREDICT

In [ ]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}
sampple_text = dataset_mail["test"][0]["article"]
reference = dataset_mail["test"][0]["highlights"]
pipe = pipeline("summarization", model="cnn_dailymail", tokenizer="tokenizer")
print("Dialogue:")
print(sample_text)
print("\nReference Summary:")
print(reference)
print("\nModel Summary:")